# Python Program for Sending from a CSV file to Kafka 

In [1]:
from time import sleep
import json
from json import dumps
from kafka import KafkaProducer
import csv
import pandas
import yaml

In [2]:
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],value_serializer=lambda x: dumps(x).encode('utf-8'))

In [5]:
#a)for simple test of sending messages to Kafka in format {"number": #} every second

for e in range(100):
    data = {'number' : e}
    producer.send('numtest', value=data)
    sleep(1)

In [6]:
#b) for sending a complete CSV file in JSON without header
with open('../PS_log-test.csv') as f:    
    for line in f:
        row = yaml.safe_load(line)
        jd = json.dumps(row)
        producer.send('csvtopic',jd)  #topic name , data

In [4]:
#c) for sending a complete CSV file in JSON with first line as header
from kafka import KafkaProducer
import time,csv


# create producer to kafka connection
producer = KafkaProducer(bootstrap_servers='localhost:9092')
# define *.csv file and a char that divide value
fname = "../PS_log-test.csv"
divider_char = ','
# open file
with open(fname) as fp:  
    # read header (first line of the input file)
    line = fp.readline()
    header = line.split(divider_char)

    #loop other data rows 
    line = fp.readline()    
    while line:
        # start to prepare data row to send
        data_to_send = ""
        values = line.split(divider_char)
        len_header = len(header)
        for i in range(len_header):
            data_to_send += "\""+header[i].strip()+"\""+":"+"\""+values[i].strip()+"\""
            if i<len_header-1 :
                data_to_send += ","
        data_to_send = "{"+data_to_send+"}"

       
        # send data via producer
        producer.send('test', bytes(data_to_send, encoding='utf-8'))
        line = fp.readline()
        # А это так))) на всякий случай
        #time.sleep(1)
producer.close()